# Оценка эффективности онлайн-маркетплейс "Заберу"

<a name="Содержание"></a> 
### Содержание:
* 0. [Описание проекта, описание данных, импорт библиотек, подключение к базе](#0.)
* 1. [Первичный осмотр](#1.)
* 2. [Подсчет затрат на каждый рекламный источник](#2.)
* 3. [Выручка с пользователей, перешедших из рекламного источника 3](#3.)
* 4. [Дневные затраты на рекламу на пользователя по источнику](#4.)

<a name="0."></a>
### 0. Описание проекта [↑](#Содержание)

Маркетплейс продает товары ручной работы от небольших производителей. «Заберу» проводит рекламные акции длительностью 1 день. Необходимо узнать, насколько эффективны такие кампании.

#### Описание данных

Таблица **visits** (лог сервера с информацией о посещениях сайта):

- `uid` — уникальный идентификатор пользователя
- `device` — категория устройства пользователя
- `startts` — дата и время начала сессии
- `endts` — дата и время окончания сессии
- `sourceid` — идентификатор рекламного источника, из которого пришел пользователь

Таблица **orders** (информация о заказах):

- `uid` — уникальный пользователя, который сделал заказ
- `buyts` — дата и время заказа
- `revenue` — выручка Яндекс.Афиши с этого заказа

Таблица **advertisment_costs** (информация о затратах на маркетинг):

- `sourceid` — идентификатор рекламного источника
- `dt` — дата
- `costs` — затраты на этот рекламный источник в этот день

#### Импорт библиотек, подключение к базе

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [3]:
db_config = {'user': 'praktikum_student',         # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # порт подключения
             'db': 'data-analyst-final-project-db'}          # название базы данных

In [4]:
# Формируем строку соединения с БД.
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                             db_config['pwd'],
                                                             db_config['host'],
                                                             db_config['port'],
                                                             db_config['db'])
# Подключаемся к БД.
engine = create_engine(connection_string, connect_args={'sslmode':'require'}, echo=True)

<a name="1."></a>
### 1. Первичный осмотр [↑](#Содержание)

In [79]:
query = ''' 
        SELECT
            *
        FROM visits
        LIMIT 3;
        '''
visits = pd.io.sql.read_sql(query, con = engine)

query = ''' 
        SELECT
            *
        FROM orders
        LIMIT 3;
        '''
orders = pd.io.sql.read_sql(query, con = engine)

query = ''' 
        SELECT
            *
        FROM advertisment_costs
        LIMIT 3;
        '''
advertisment_costs = pd.io.sql.read_sql(query, con = engine)

2020-03-16 12:52:16,592 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-03-16 12:52:16,594 INFO sqlalchemy.engine.base.Engine {'name': ' \n        SELECT\n            *\n        FROM visits\n        LIMIT 3;\n        '}
2020-03-16 12:52:16,625 INFO sqlalchemy.engine.base.Engine  
        SELECT
            *
        FROM visits
        LIMIT 3;
        
2020-03-16 12:52:16,626 INFO sqlalchemy.engine.base.Engine {}
2020-03-16 12:52:16,659 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-03-16 12:52:16,660 INFO sqlalchemy.engine.base.Engine {'name': ' \n        SELECT\n            *\n        FROM orders\n        LIMIT 3;\n        '}
2020-03-16 12:52:16,691 INFO sqlalchemy.engine.base.Engine  
        SELECT
            *
        

In [78]:
visits

,id,uid,device,endts,sourceid,startts,id,buyts,revenue,uid
0,0,16879256277535980062,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00,1,2017-06-01 00:10:00,$17.00,10329302124590727494
1,0,16879256277535980062,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00,2,2017-06-01 00:25:00,$0.55,11627257723692907447
2,0,16879256277535980062,touch,2017-12-20 17:38:00,4,2017-12-20 17:20:00,3,2017-06-01 00:27:00,$0.37,17903680561304213844


In [74]:
orders

,id,buyts,revenue,uid
0,1,2017-06-01 00:10:00,$17.00,10329302124590727494
1,2,2017-06-01 00:25:00,$0.55,11627257723692907447
2,3,2017-06-01 00:27:00,$0.37,17903680561304213844


In [75]:
advertisment_costs

,id,sourceid,dt,costs
0,1,1,2017-06-01,$75.20
1,2,1,2017-06-02,$62.25
2,3,1,2017-06-03,$36.53


<a name="2."></a>
### 2. Подсчет затрат на каждый рекламный источник [↑](#Содержание)

In [80]:
query = ''' 
        SELECT
            sourceid,
            SUM(costs)
        FROM 
            advertisment_costs
        GROUP BY 
            sourceid
        ORDER BY 
            SUM(costs) DESC;
        '''
df = pd.io.sql.read_sql(query, con = engine)
df

2020-03-16 12:59:27,679 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-03-16 12:59:27,680 INFO sqlalchemy.engine.base.Engine {'name': ' \n        SELECT\n            sourceid,\n            SUM(costs)\n        FROM \n            advertisment_costs\n        GROUP BY \n            sourceid\n        ORDER BY \n            SUM(costs) DESC;\n        '}
2020-03-16 12:59:27,714 INFO sqlalchemy.engine.base.Engine  
        SELECT
            sourceid,
            SUM(costs)
        FROM 
            advertisment_costs
        GROUP BY 
            sourceid
        ORDER BY 
            SUM(costs) DESC;
        
2020-03-16 12:59:27,715 INFO sqlalchemy.engine.base.Engine {}


,sourceid,sum
0,3,"$141,321.63"
1,4,"$61,073.60"
2,5,"$51,757.10"
3,2,"$42,806.04"
4,1,"$20,833.27"
5,10,"$5,822.49"
6,9,"$5,517.49"


По сумме затрат значительно лидирует **3 источник**. Разница в затратах на источники доходит **почти до 3 порядков**.

<a name="3."></a>
### 3. Выручка с пользователей, перешедших из рекламного источника 3 [↑](#Содержание)

Посчитаем cуммарную выручку, максимальную и минимальную выручки от заказов, сделанных с настольных ПК пользователями, которые хотя бы раз перешли из рекламного источника 3.

In [89]:
query = '''
        SELECT 
            SUM(orders.revenue),
            MIN(orders.revenue), 
            MAX(orders.revenue)
        FROM 
            orders
        WHERE 
            orders.uid IN (SELECT 
                               DISTINCT visits.uid
                           FROM 
                               visits
                           WHERE 
                               visits.device = 'desktop' AND 
                               visits.sourceid = '3');
        '''

df = pd.io.sql.read_sql(query, con = engine)
df

2020-03-16 13:08:23,333 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-03-16 13:08:23,333 INFO sqlalchemy.engine.base.Engine {'name': "\n        SELECT \n            SUM(orders.revenue),\n            MIN(orders.revenue), \n            MAX(orders.revenue)\n        FROM \n            o ... (197 characters truncated) ...               WHERE \n                               visits.device = 'desktop' AND \n                               visits.sourceid = '3');\n        "}
2020-03-16 13:08:23,365 INFO sqlalchemy.engine.base.Engine 
        SELECT 
            SUM(orders.revenue),
            MIN(orders.revenue), 
            MAX(orders.revenue)
        FROM 
            orders
        WHERE 
            orders.uid IN (SELECT 
                               DISTINCT visits.uid
                           FROM 
                               visits
               

,sum,min,max
0,"$106,058.86",$0.00,"$2,633.28"


Выручка с устройств Desktop не покрывает затрат на 3 рекламный источник.

<a name="4."></a>
### 4. Дневные затраты на рекламу на пользователя по источнику [↑](#Содержание)

In [91]:
query = '''
        SELECT 
            advertisment_costs.dt,
            advertisment_costs.sourceid,
            advertisment_costs.costs,
            visits_grouped.cnt_users,
            advertisment_costs.costs / visits_grouped.cnt_users AS metric
        FROM 
            advertisment_costs 
           
        JOIN (
              SELECT 
                  DATE_TRUNC('DAY', visits.startts::timestamp) AS dt, 
                  visits.sourceid, 
                  COUNT(DISTINCT(visits.uid)) AS cnt_users
              FROM 
                  visits
              GROUP BY 
                  dt,
                  visits.sourceid
                               
                ) AS visits_grouped ON 
                  (advertisment_costs.sourceid = visits_grouped.sourceid AND 
                   DATE_TRUNC('day', advertisment_costs.dt::timestamp) = visits_grouped.dt)
        ORDER BY
                visits_grouped.dt
        LIMIT 30;
        '''
df = pd.io.sql.read_sql(query, con = engine)
df

2020-03-16 13:09:12,122 INFO sqlalchemy.engine.base.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2020-03-16 13:09:12,123 INFO sqlalchemy.engine.base.Engine {'name': "\n        SELECT \n            advertisment_costs.dt,\n            advertisment_costs.sourceid,\n            advertisment_costs.costs,\n            v ... (677 characters truncated) ... TRUNC('day', advertisment_costs.dt::timestamp) = visits_grouped.dt)\n        ORDER BY\n                visits_grouped.dt\n        LIMIT 30;\n        "}
2020-03-16 13:09:12,153 INFO sqlalchemy.engine.base.Engine 
        SELECT 
            advertisment_costs.dt,
            advertisment_costs.sourceid,
            advertisment_costs.costs,
            visits_grouped.cnt_users,
            advertisment_costs.costs / visits_grouped.cnt_users AS metric
        FROM 
            advertisment_costs 
           
        JOIN (
              SELECT

,dt,sourceid,costs,cnt_users,metric
0,2017-06-01,1,$75.20,59,$1.27
1,2017-06-01,2,$132.56,79,$1.67
2,2017-06-01,3,$349.38,182,$1.91
3,2017-06-01,4,$76.74,138,$0.55
4,2017-06-01,5,$73.24,114,$0.64
5,2017-06-01,9,$13.06,51,$0.25
6,2017-06-01,10,$15.08,9,$1.67
7,2017-06-02,1,$62.25,50,$1.24
8,2017-06-02,2,$132.38,91,$1.45
9,2017-06-02,3,$314.22,179,$1.75


Пользователи перешедшие из 3 рекламного источника являются самыми дорогими. Разница с самыми "дешевыми" достигает 4-6 раз. Однако, источник 3 привлекает наибольшее количество пользователей.

Можно порекомендовать сократить расходы или полностью перенаправить рекламный бюджет на другие источники, если целью является сокращение маркетинговых расходов. В случае, если основная цель привлечение наибольшего количества пользователей - 3 источник показывает лучшие результаты.